In [1]:
File_Location = str("D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\")

In [2]:
import pandas as pd
import cv2
import pyautogui
import keyboard
import cv2
import time
import pyperclip
import os

In [3]:
df_Excel_Raw = pd.read_excel(File_Location + str("Canada_Hosp1_COVID_InpatientData.xlsx"))

comorbidities_other = df_Excel_Raw['comorbidities_other']
comorbidities_other = comorbidities_other.dropna()
comorbidities_other = comorbidities_other.reset_index(drop=True)
## comorbidities_other = comorbidities_other[388:]
## comorbidities_other = comorbidities_other.reset_index(drop=True)
comorbidities_other

0      GERD (gastroesophageal reflux disease), Dement...
1      Closed fracture of fifth metatarsal bone, Dive...
2      Parkinson's Disease, Back pain, Benign Prostat...
3      TAVI, Atrial fibrillation, Gastric Reflux, Pac...
4      Coronary artery disease with previous bypass s...
                             ...                        
399    Depression, GERD (gastroesophageal reflux dise...
400                                         Dyslipidemia
401    Arthritis, High cholesterol, Hypothyroidism, V...
402    GERD (gastroesophageal reflux disease), High c...
403    Dermatitis, Hyponatremia, Irritable bowel synd...
Name: comorbidities_other, Length: 404, dtype: object

In [36]:
print(comorbidities_other[0])

GERD (gastroesophageal reflux disease), Dementia Depression, Barrett esophagus, Pneumonia, Schizophrenia, Hiatus Hernia, Reflux esophagitis


In [ ]:
## 101-200     0-101  27-128

In [4]:
## Save the txt file that we have processed.

comorbidities_other.to_csv('D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\comorbidities_other.txt', sep='\t', header=False, index=False)

In [48]:
def find_small_image_coordinates(big_image_path, small_image_path):
    # Load the bigger image and the template (smaller image)
    big_image = cv2.imread(big_image_path)
    small_image = cv2.imread(small_image_path)
    
    if big_image is None:
        print(f"Error: Could not load big image from path: {big_image_path}")
        return None

    if small_image is None:
        print(f"Error: Could not load small image from path: {small_image_path}")
        return None
    
    # Get the width and height of the template
    w, h = small_image.shape[:-1]

    # Perform template matching
    res = cv2.matchTemplate(big_image, small_image, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # Get the top left coordinates of the matched area
    top_left = max_loc

    # Calculate the bottom right coordinates
    bottom_right = (top_left[0] + h, top_left[1] + w)

    return top_left, bottom_right

In [49]:
big_image_path = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Screenshot.png'
small_image_path = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Test Pipeline.png'
Test_Pipeline = find_small_image_coordinates(big_image_path, small_image_path)
Center_Test_Pipeline = [round((Test_Pipeline[0][0]+Test_Pipeline[1][0])/2), round((Test_Pipeline[0][1]+Test_Pipeline[1][1])/2)]
print(Center_Test_Pipeline)

[158, 833]


In [ ]:
# download coordinate [308,833]

In [50]:
big_image_path = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Screenshot.png'
small_image_path = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Type Content.png'
Type_Content = find_small_image_coordinates(big_image_path, small_image_path)
print(Type_Content)

((292, 133), (1840, 374))


In [8]:
big_image_path = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Screenshot.png'
small_image_path = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Annotation.png'
Annotation = find_small_image_coordinates(big_image_path, small_image_path)
print(Annotation)

((75, 931), (1832, 1153))


In [51]:
pyautogui.hotkey('alt', 'tab')

text_result = []

# Function to click on specific coordinates
def click_coordinates(x, y):
    pyautogui.click(x, y)
    time.sleep(1)

# Function to press ca key
def press_key(key):
    pyautogui.press(key)
    time.sleep(0.4)
    
def release_mouse_button():
    pyautogui.mouseUp()  # Release the mouse button
    time.sleep(1)

# Defining the function to click and hold at specific coordinates
def click_and_hold_coordinates(x, y):
    pyautogui.moveTo(x, y)  # Move the mouse to the specific coordinates
    pyautogui.mouseDown()  # Press the mouse button
    time.sleep(1)


try:
    
        for index, row in comorbidities_other.iteritems():
            # Looping through each row of comorbidities_other Series

            # Copy the content of the current row
            pyperclip.copy(row)
    
            # Performing the sequence of actions
            click_and_hold_coordinates(292, 133)
            pyautogui.moveTo(1840, 374)  # Move the mouse to the new coordinates while holding the button
            release_mouse_button()  # Release the mouse button
            press_key('delete')
    
            # Paste the content
            pyautogui.hotkey('ctrl', 'v')  # This line simulates CTRL+V, pasting the text
            time.sleep(0.4)  # Waiting 0.4 seconds before the next action
    
            click_coordinates(158, 833)
        
            click_coordinates(308, 833)
            
            if keyboard.is_pressed('='):  # Check if '=' is pressed
                print("Terminating script.")
                break  # Exit the loop and terminate the script

            
except KeyboardInterrupt:  # Allow termination by pressing CTRL+C
        
        print("Script terminated by user.")

# Function to listen for the '=' key to terminate the script
keyboard.add_hotkey('=', lambda: exit())

Terminating script.


<function keyboard.add_hotkey.<locals>.remove_()>

In [4]:
# Function to load data from a single file
def load_data(file_path):
    with open(file_path, 'r') as file:
        data = file.readlines()
    return data

def load_files_to_dataframe(directory_path):
    # Creating an empty DataFrame to store the data
    df = pd.DataFrame()

    # Reading data from 'resultFile.txt' and adding it to the DataFrame first
    result_file_path = os.path.join(directory_path, "resultFile.txt")
    if os.path.exists(result_file_path):
        result_data = load_data(result_file_path)
        result_df = pd.DataFrame(result_data, columns=['Text'])
        result_df['File'] = "resultFile.txt"
        df = pd.concat([result_df, df]).reset_index(drop=True)
    else:
        print(f"File not found: {result_file_path}")

    # Iterating over each file and loading the data into the DataFrame
    for i in range(1, 101):  # 1 to 100 inclusive
        file_path = os.path.join(directory_path, f"resultFile ({i}).txt")

        if os.path.exists(file_path):
            data = load_data(file_path)

            # Assuming each line in the file is a separate record
            temp_df = pd.DataFrame(data, columns=['Text'])

            # Adding a column to identify the file
            temp_df['File'] = f"resultFile ({i}).txt"

            # Appending data to the main DataFrame
            df = df.append(temp_df, ignore_index=True)
        else:
            print(f"File not found: {file_path}")

    # Returning the DataFrame containing data from all files
    return df

In [5]:
directory_path_1 = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\0 - 100'
directory_path_2 = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\101 - 201'
directory_path_3 = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\202 - 302'
directory_path_4 = 'D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\303 - 403'

df_1 = load_files_to_dataframe(directory_path_1)
df_2 = load_files_to_dataframe(directory_path_2)
df_3 = load_files_to_dataframe(directory_path_3)
df_4 = load_files_to_dataframe(directory_path_4)

df = pd.concat([df_1, df_2, df_3, df_4], ignore_index=True)

C:\Users\harve\AppData\Local\Temp\ipykernel_21556\1332376679.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\1332376679.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\1332376679.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\1332376679.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
C:\Users\harve\A

In [6]:
df

Text                  File
0    {"text":"GERD (gastroesophageal reflux disease...        resultFile.txt
1    {"text":"Closed fracture of fifth metatarsal b...    resultFile (1).txt
2    {"text":"Parkinson's Disease, Back pain, Benig...    resultFile (2).txt
3    {"text":"TAVI, Atrial fibrillation, Gastric Re...    resultFile (3).txt
4    {"text":"Coronary artery disease with previous...    resultFile (4).txt
..                                                 ...                   ...
399  {"text":"Depression, GERD (gastroesophageal re...   resultFile (96).txt
400  {"text":"Dyslipidemia","entities":{"Disease":[...   resultFile (97).txt
401  {"text":"Arthritis, High cholesterol, Hypothyr...   resultFile (98).txt
402  {"text":"GERD (gastroesophageal reflux disease...   resultFile (99).txt
403  {"text":"Dermatitis, Hyponatremia, Irritable b...  resultFile (100).txt

[404 rows x 2 columns]

In [7]:
def add_pref_columns(df, column_name='Text'):
    # Iterating over each row in the DataFrame
    for index, row in df.iterrows():
        text = row[column_name]
        
        # Finding all occurrences of the pattern "PREF":"<content>"
        start_idx = text.find('"PREF":"')
        
        while start_idx != -1:
            start_idx += 8  # Skipping the '"PREF":"' part
            end_idx = text.find('"', start_idx)
            
            if end_idx != -1:
                pref_value = text[start_idx:end_idx]
                
                # Checking if the column already exists, if not then creating it
                if pref_value not in df.columns:
                    df[pref_value] = 0
                
                # Incrementing the value in the corresponding column
                df.at[index, pref_value] += 1
            
            # Looking for the next occurrence
            start_idx = text.find('"PREF":"', end_idx)
    
    return df

# Test the function
data = df['Text']

df_PREF = pd.DataFrame(data)

# Calling the function and printing the resulting DataFrame
df_PREF = add_pref_columns(df_PREF)
print(df_PREF)

C:\Users\harve\AppData\Local\Temp\ipykernel_21556\3381632470.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[pref_value] = 0
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\3381632470.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[pref_value] = 0
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\3381632470.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

                                                  Text  Acid Reflux  \
0    {"text":"GERD (gastroesophageal reflux disease...            6   
1    {"text":"Closed fracture of fifth metatarsal b...            0   
2    {"text":"Parkinson's Disease, Back pain, Benig...            3   
3    {"text":"TAVI, Atrial fibrillation, Gastric Re...            3   
4    {"text":"Coronary artery disease with previous...            0   
..                                                 ...          ...   
399  {"text":"Depression, GERD (gastroesophageal re...            6   
400  {"text":"Dyslipidemia","entities":{"Disease":[...            0   
401  {"text":"Arthritis, High cholesterol, Hypothyr...            0   
402  {"text":"GERD (gastroesophageal reflux disease...            6   
403  {"text":"Dermatitis, Hyponatremia, Irritable b...            0   

     closed fracture of fifth metatarsal  Diverticulosis  Colon spasm  \
0                                      0               0            0   
1

C:\Users\harve\AppData\Local\Temp\ipykernel_21556\3381632470.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[pref_value] = 0
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\3381632470.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[pref_value] = 0
C:\Users\harve\AppData\Local\Temp\ipykernel_21556\3381632470.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

In [34]:
df_PREF

Text  Acid Reflux  \
0    {"text":"GERD (gastroesophageal reflux disease...            6   
1    {"text":"Closed fracture of fifth metatarsal b...            0   
2    {"text":"Parkinson's Disease, Back pain, Benig...            3   
3    {"text":"TAVI, Atrial fibrillation, Gastric Re...            3   
4    {"text":"Coronary artery disease with previous...            0   
..                                                 ...          ...   
399  {"text":"Depression, GERD (gastroesophageal re...            6   
400  {"text":"Dyslipidemia","entities":{"Disease":[...            0   
401  {"text":"Arthritis, High cholesterol, Hypothyr...            0   
402  {"text":"GERD (gastroesophageal reflux disease...            6   
403  {"text":"Dermatitis, Hyponatremia, Irritable b...            0   

     closed fracture of fifth metatarsal  Diverticulosis  Colon spasm  \
0                                      0               0            0   
1                                      3               3            3   
2                                      0               0            0   
3                                      0               0            0   
4                                      0               0            0   
..                                   ...             ...          ...   
399                                    0               0            0   
400                                    0               0            0   
401                                    0               0            0   
402                                    0               0            0   
403                                    0               0            3   

     Vertigo  parkinson's  Backache  BPH  Cholecystitis chronic  ...  \
0          0            0         0    0                      0  ...   
1          3            0         0    0                      0  ...   
2          0            3         3    3                      3  ...   
3          0            0         0    0                      0  ...   
4          0            0         0    0                      0  ...   
..       ...          ...       ...  ...                    ...  ...   
399        0            0         0    0                      0  ...   
400        0            0         0    0                      0  ...   
401        3            0         0    0                      0  ...   
402        0            0         0    0                      0  ...   
403        0            0         0    0                      0  ...   

     Opisthocranion  Colonoscopy  Aneurysm;artery;cerebral  \
0                 0            0                         0   
1                 0            0                         0   
2                 0            0                         0   
3                 0            0                         0   
4                 0            0                         0   
..              ...          ...                       ...   
399               0            0                         0   
400               0            0                         0   
401               0            0                         0   
402               0            0                         0   
403               0            0                         0   

     Bilateral inguinal hernia  Schizoaffective Disorder  Acoustic Tumor  \
0                            0                         0               0   
1                            0                         0               0   
2                            0                         0               0   
3                            0                         0               0   
4                            0                         0               0   
..                         ...                       ...             ...   
399                          0                         0               0   
400                          0                         0               0   
401                       

In [38]:
# Defining a function to convert non-zero numbers to 1
def convert_non_zero_to_one(value):
    if isinstance(value, (int, float)) and value != 0:
        return 1
    return value

# Applying the function to all columns except 'Text'
columns_except_text = [col for col in df_PREF.columns if col != 'Text']
df_PREF[columns_except_text] = df_PREF[columns_except_text].applymap(convert_non_zero_to_one)

# Printing the modified DataFrame
df_PREF

Text  Acid Reflux  \
0    {"text":"GERD (gastroesophageal reflux disease...            1   
1    {"text":"Closed fracture of fifth metatarsal b...            0   
2    {"text":"Parkinson's Disease, Back pain, Benig...            1   
3    {"text":"TAVI, Atrial fibrillation, Gastric Re...            1   
4    {"text":"Coronary artery disease with previous...            0   
..                                                 ...          ...   
399  {"text":"Depression, GERD (gastroesophageal re...            1   
400  {"text":"Dyslipidemia","entities":{"Disease":[...            0   
401  {"text":"Arthritis, High cholesterol, Hypothyr...            0   
402  {"text":"GERD (gastroesophageal reflux disease...            1   
403  {"text":"Dermatitis, Hyponatremia, Irritable b...            0   

     closed fracture of fifth metatarsal  Diverticulosis  Colon spasm  \
0                                      0               0            0   
1                                      1               1            1   
2                                      0               0            0   
3                                      0               0            0   
4                                      0               0            0   
..                                   ...             ...          ...   
399                                    0               0            0   
400                                    0               0            0   
401                                    0               0            0   
402                                    0               0            0   
403                                    0               0            1   

     Vertigo  parkinson's  Backache  BPH  Cholecystitis chronic  ...  \
0          0            0         0    0                      0  ...   
1          1            0         0    0                      0  ...   
2          0            1         1    1                      1  ...   
3          0            0         0    0                      0  ...   
4          0            0         0    0                      0  ...   
..       ...          ...       ...  ...                    ...  ...   
399        0            0         0    0                      0  ...   
400        0            0         0    0                      0  ...   
401        1            0         0    0                      0  ...   
402        0            0         0    0                      0  ...   
403        0            0         0    0                      0  ...   

     Opisthocranion  Colonoscopy  Aneurysm;artery;cerebral  \
0                 0            0                         0   
1                 0            0                         0   
2                 0            0                         0   
3                 0            0                         0   
4                 0            0                         0   
..              ...          ...                       ...   
399               0            0                         0   
400               0            0                         0   
401               0            0                         0   
402               0            0                         0   
403               0            0                         0   

     Bilateral inguinal hernia  Schizoaffective Disorder  Acoustic Tumor  \
0                            0                         0               0   
1                            0                         0               0   
2                            0                         0               0   
3                            0                         0               0   
4                            0                         0               0   
..                         ...                       ...             ...   
399                          0                         0               0   
400                          0                         0               0   
401                       

In [43]:
excel_file_path = "D:\\Document\\Research\\Hospital Occupancy Monitoring Project\\Data\\Comorbidities_PREF.xlsx"

# Exporting the DataFrame to an Excel file
df_PREF.to_excel(excel_file_path, index=False, engine='openpyxl')

# Printing a message to indicate the export was successful
print(f"DataFrame has been exported to: {excel_file_path}")

DataFrame has been exported to: D:\Document\Research\Hospital Occupancy Monitoring Project\Data\Comorbidities_PREF.xlsx


In [32]:
df_new = df_PREF.iloc[0:,1:].T
df_new['Sum'] = 0
df_new

for i in range(len(df_new)):
    df_new['Sum'][i] = sum(df_new.iloc[i,0:])
    
new_i = df_new['Sum'].sort_values(ascending=True)

In [33]:
new_i[700:]

Before                      18
Mild cognitive disorder     18
Chronic Renal Disease       18
Chronic                     18
CHF NOS                     21
Heart Attack                24
Nephrolith                  24
COPD NOS                    24
Cataract                    24
Peptic Ulcer                24
Continuous CPAP             26
DVT NOS                     27
Fit(s)                      27
Spinal Stenoses             27
Fracture                    30
Pneumonia                   33
Ferropenic                  33
Diverticulosis              36
Embolism lung               36
Glaucoma                    36
Thyroid                     45
Sleep Apnea                 45
Anxiety                     48
Urinary infection           57
Coronary Disease            57
Obstructive Apnea           60
AFib                        66
Arthritis                   66
Benign prostatic hyper.     81
Osteoporoses                84
BPH                         84
Depression                  84
Gout NOS